# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Querying data through Apache Cassandra

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events
    WITH REPLICATION =
    { 'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

## Query 1
Find the `artist`, `song title` and `song's length` in the music app history that was heard during `sessionId = 338`, and `itemInSession = 4`

In [8]:
query = "CREATE TABLE IF NOT EXISTS artists_songs_info"
query += "(sessionId int, itemInSession int, artist varchar, song varchar, \
           length decimal, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)           

In [9]:
file = 'event_datafile_new.csv'

with open(file) as csv_file:
    data = csv.DictReader(csv_file)
    # Assign the INSERT statements into the `query` variable
    query = "INSERT INTO artists_songs_info \
            (sessionId, itemInSession, artist, song, length)"
    query += "VALUES (%s, %s, %s, %s, %s)"
    for row in data:
        length = float(row['length'])
        sessionId = int(row['sessionId'])
        itemInSession = int(row['itemInSession'])
        # Option 1: Assign the column elements to the INSERT statement
        session.execute(query, (sessionId, itemInSession, row['artist'], row['song'], length))
        
        # Option 2: Only insert if value matches
#         if sessionId == 338 and itemInSession == 4:
#             session.execute(query, (sessionId, itemInSession, row['artist'], row['song'], length))

#### SELECT to verify that the data have been inserted into each table

In [10]:
query = 'SELECT * FROM artists_songs_info limit 1'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,sessionid,iteminsession,artist,length,song
0,23,0,Regina Spektor,191.08526,The Calculation (Album Version)


In [11]:
# Find the artist, title and song's length in the data during sessionId = 338, and itemInSession = 4
query = 'SELECT artist, song, length \
         FROM artists_songs_info \
         WHERE sessionId = 338 and itemInSession = 4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


## Query 2
Find the `name of artist`, `song` (sorted by itemInSession) and `user` (first and last name) for `userid = 10`, `sessionid = 182`

In [30]:
query = "CREATE TABLE IF NOT EXISTS artists_songs_users_info"
query += "(userId int, sessionId int, itemInSession int, artist varchar, song varchar, \
           firstName varchar, lastName varchar, PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [31]:
file = 'event_datafile_new.csv'

with open(file) as csv_file:
    data = csv.DictReader(csv_file)
    # Assign the INSERT statements into the `query` variable
    query = "INSERT INTO artists_songs_users_info \
            (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
    query += "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    for row in data:
        userId = int(row['userId'])
        itemInSession = int(row['itemInSession'])
        sessionId = int(row['sessionId'])
        # Option 1: Assign the column elements to the INSERT statement
        session.execute(query, (userId, sessionId, itemInSession, row['artist'], 
                                row['song'], row['firstName'], row['lastName']))
        
        # Option 2: Only insert if value matches
#         if userId == 10 and sessionId == 182:
#             session.execute(query, (userId, sessionId, itemInSession, row['artist'], 
#                                 row['song'], row['firstName'], row['lastName']))

#### SELECT to verify that the data have been inserted into each table

In [32]:
query = 'SELECT * FROM artists_songs_users_info limit 1'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,userid,sessionid,iteminsession,artist,firstname,lastname,song
0,23,177,2,Dwight Yoakam,Morris,Gilmore,You're The One


In [33]:
# Find the artist, song and user (first and last name) for userid = 10, sessionid = 182
query = 'SELECT artist, song, firstName, lastName \
         FROM artists_songs_users_info \
         WHERE userId = 10 AND sessionId = 182'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3
Find every `user name` (first and last) in the data who listened to the song `'All Hands Against His Own'`

In [19]:
query = "CREATE TABLE IF NOT EXISTS users_songs_table"
query += "(song varchar, userId int, firstName varchar, lastName varchar, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)       

In [20]:
file = 'event_datafile_new.csv'

with open(file) as csv_file:
    data = csv.DictReader(csv_file)
    # Assign the INSERT statements into the `query` variable
    query = "INSERT INTO users_songs_table (song, userId, firstName, lastName)"
    query += "VALUES (%s, %s, %s, %s)"
    for row in data:
        userId = int(row['userId'])
        # Option 1: Assign the column elements to the INSERT statement
        session.execute(query, (row['song'], userId, row['firstName'], row['lastName']))
        
        # Option 2: Only insert if value matches
#         if row['song'] == 'All Hands Against His Own':
#             session.execute(query, (row['song'], userId, row['firstName'], row['lastName']))

#### SELECT to verify that the data have been inserted into each table

In [21]:
query = 'SELECT * FROM users_songs_table limit 1'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,song,userid,firstname,lastname
0,Wonder What's Next,49,Chloe,Cuevas


In [22]:
# Find every user (first and last) in the data who listened to the song 'All Hands Against His Own'
query = "SELECT firstName, lastName \
         FROM users_songs_table \
         WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [29]:
query = "DROP TABLE artists_songs_info"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE artists_songs_users_info"
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE users_songs_table"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()